In [1]:
import pyspark as ps
import pandas as pd
import re

In [66]:
#df = pd.read_csv('work/data/articles1.csv')

In [74]:
#df = df['content']

In [75]:
#df.to_csv('work/data/art1.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [2]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('news')
         .getOrCreate()
        )

In [7]:
#df = spark.read.load('work/data/art1.parquet',
     inferSchema='true', header='true')

In [8]:
#spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [9]:
#df.columns

['id', 'content']

In [14]:
from pyspark.sql.functions import *

In [27]:
#df = df.select(df['id'], split(df['content'], '[?.,—!;:\’\'"“”\(\)]').alias('content'))

In [40]:
#df = df.select(df['id'], explode(col('content')).alias('content'))

In [43]:
#df = df.select(df['id'], trim(col('content')).alias('content'))

In [44]:
#df = df.select(df['id'], split(col('content'), ' ').alias('content'))

In [3]:
sc = spark.sparkContext

In [4]:
file_rdd = sc.textFile('work/data/art1.csv')

In [5]:
bigrams = file_rdd.flatMap(lambda line: re.split('[?.,—!;:\’\'"“”\(\)]', line)) \
                   .map(lambda line: line.strip().split(" ")) \
                   .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))
#bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [6]:
bigrams.count()

28452375

In [9]:
bigrams = bigrams.filter(lambda x: (len(x[0])>0)&(len(x[1])>0))

In [11]:
# import the many data types
from pyspark.sql.types import StructType, StringType, StructField

# create a schema of your own
schema = StructType( [
    StructField('w1',StringType(),True),
    StructField('w2',StringType(),True)] )

In [35]:
df = spark.createDataFrame(bigrams, schema)

In [36]:
df = df.select(col('w1'), col('w2'), concat(col('w1'), lit(' '), col('w2')).alias('phrase'))

In [37]:
df.distinct()

DataFrame[w1: string, w2: string, phrase: string]

[('Congressional', 'Republicans'),
 ('Republicans', 'have'),
 ('have', 'a'),
 ('a', 'new'),
 ('new', 'fear')]